In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense, Flatten, Reshape, BatchNormalization
from keras.layers import Conv1D, UpSampling1D, MaxPooling1D, AveragePooling1D
from keras.models import Model, Sequential
from keras.layers.advanced_activations import ReLU
from keras.optimizers import RMSprop
import tensorflow as tf


C_PATH = "data/merged_chest_fltr.pkl"
W1_PATH = "data/subj_merged_bvp_w.pkl"
W2_PATH = "data/subj_merged_eda_temp_w.pkl"
feat_sf700 = ['ecg', 'emg', 'eda', 'temp', 'resp']
feat_sf64 = ['bvp']
feat_sf4 = ['w_eda', 'w_temp']
feat_all = ['ecg', 'emg', 'c_eda', 'c_temp', 'resp', 'bvp', 'w_eda', 'w_temp']
sf_chest = 700 #sampling frequency for measurements collected from chest device
sf_BVP = 64
sf_EDA = 4
sf_TEMP = 4

window = 0.25 # sampling window


In [2]:
from sklearn.metrics import accuracy_score, f1_score

In [3]:
df = pd.read_pickle('subj_merged.pkl')
df.columns
#feat_list = [ 'ecg', 'emg', 'c_eda', 'c_temp', 'resp', 'bvp', 'w_eda', 'w_temp']
#feat_list = [ 'ecg', 'emg', 'c_eda', 'c_temp', 'resp']
feat_list = ['bvp', 'w_eda', 'w_temp']
df = df[df["c_temp"]>0]
df = df[df["label"].isin([1,2,3])]

In [23]:
class cnn_model:
    def __init__(self, **kwargs):
        self.df_c = pd.read_pickle(C_PATH)
        self.df_w1 = pd.read_pickle(W1_PATH)
        self.df_w2 = pd.read_pickle(W2_PATH)
        self.df_w1 = self.df_w1[self.df_w1["label"].isin([1,2,3])]
        self.df_w2 = self.df_w2[self.df_w2["label"].isin([1,2,3])]
        
        self.batch_size = int(sf_chest*window) 
        self.batch_size_bvp =  int(sf_BVP*window) 
        self.batch_size_eda =  int(sf_EDA*window) 
        self.batch_size_temp =  int(sf_TEMP*window) 

        self.ids = self.df_c["sid"].unique().astype(int)
        self.K = len(self.df_c["label"].unique())
        
    def one_hot_enc(self, r, k):
        new_r = np.zeros((r.shape[0],k))
        for i, val in enumerate(r):
            new_r[i, val-1] = 1

        return new_r
    
    def get_data(self, test_id, v_batch_size, v_feat_list, df):
        
        cnt=0
        
        for j in self.ids:
            df_s = df[df["sid"] == j]

            n = (len(df_s)//v_batch_size)*v_batch_size
            df_s = df_s[:n]
            s = StandardScaler().fit_transform(df_s[v_feat_list])
            s = s.reshape(int(s.shape[0]/v_batch_size), s.shape[1],  v_batch_size)

            lbl_m = np.zeros((s.shape[0],1))
            lbl = df_s["label"].values.astype(int)
            for i in range(s.shape[0]):
                lbl_m[i] = int((stats.mode(lbl[i * v_batch_size : (i + 1) * v_batch_size - 1]))[0].squeeze())
            y_k = lbl_m.astype(int)
            s_y = self.one_hot_enc(lbl_m.astype(int), self.K).astype(int)
            #print("subject ", j)
            #print(s.shape, s_y.shape)
            if j==test_id:
                x_test = s
                y_test = s_y
                yk_test = y_k
            else:
                if cnt:
                    merged = np.concatenate((merged, s), axis=0)
                    merged_y = np.concatenate((merged_y, s_y), axis=0)
                    merged_yk = np.concatenate((merged_yk, y_k), axis=0)
                else:
                    merged = s
                    merged_y = s_y
                    merged_yk = y_k
                cnt +=1


        print ("merged train:", merged.shape, merged_y.shape)
        print ("merged test :", x_test.shape, y_test.shape)
        return merged, merged_y, x_test, y_test, merged_yk, yk_test

    # train and store autoencoder model for chest modalities
    def train_model_c(self):   
        # leave one out method
        scores = []

        for sid in self.ids:
            x_train, y_train, x_test, y_test, yk, yk_test = self.get_data (test_id =sid, 
                                                                       v_batch_size=sf_chest, 
                                                                       v_feat_list=feat_sf700, 
                                                                       df=self.df_c)

            model = self.cnn_model_chest(v_batch_size=sf_chest, n_feat=len(feat_sf700))
            model.compile(optimizer=RMSprop(lr=0.00025), loss="categorical_crossentropy")
            history = model.fit(x_train, y_train, epochs=10)
            m_name = "trained_models/c/cnn_loso"+str(sid)+".h5"

            model.save(m_name)
            print("saved ", m_name)
            


    def train_model_all(self, v_df):   
            # leave one out method
            scores = []

            for sid in self.ids:
                x_train, y_train, x_test, y_test, yk, yk_test = self.get_data (test_id =sid, 
                                                                           v_batch_size=4, 
                                                                           v_feat_list=feat_all, 
                                                                           df=v_df)

                model = self.cnn_model_all(v_batch_size=4, n_feat=len(feat_all))
                model.compile(optimizer=RMSprop(lr=0.00025), loss="categorical_crossentropy")
                history = model.fit(x_train, y_train, epochs=10)
                m_name = "trained_models/c/cnn_all_loso"+str(sid)+".h5"

                model.save(m_name)
                print("saved ", m_name)
            
            
    def cnn_model_w1(self, v_batch_size, n_feat):
    
        input_sig = Input(shape=(n_feat, v_batch_size))
        x = Conv1D(v_batch_size,6, activation='relu', padding='same')(input_sig)
        x1 = BatchNormalization()(x)
        x2 = Conv1D(v_batch_size,3, activation='relu', padding='same')(x1)
        flat = Flatten()(x2)

        encoded = Dense(40, activation='relu')(flat)
        cls = Dense(3, activation='softmax')(encoded)

        model= Model(input_sig, cls)

        return model
    
    def cnn_model_w2(self, v_batch_size, n_feat):
        
        input_sig = Input(shape=(n_feat, v_batch_size))
        x = Conv1D(v_batch_size,4, activation='relu', padding='same')(input_sig)

        x1 = BatchNormalization()(x)
        flat = Flatten()(x1)
        encoded = Dense(4, activation='relu')(flat)
        cls = Dense(3, activation='softmax')(encoded)

        model= Model(input_sig, cls)

        return model   
    
    def cnn_model_chest(self, v_batch_size, n_feat):
    
        input_sig = Input(shape=(n_feat, v_batch_size))
        x = Conv1D(v_batch_size,6, activation='relu', padding='same')(input_sig)

        x1 = BatchNormalization()(x)
        x2 = Conv1D(v_batch_size,3, activation='relu', padding='same')(x1)
        flat = Flatten()(x2)
        
        encoded = Dense(80, activation='relu')(flat)
        cls = Dense(3, activation='softmax')(encoded)

        model= Model(input_sig, cls)

        return model
    
    def cnn_model_all(self, v_batch_size, n_feat):
    
        input_sig = Input(shape=(n_feat, v_batch_size))
        x = Conv1D(v_batch_size,6, activation='relu', padding='same')(input_sig)

        x1 = BatchNormalization()(x)
        x2 = Conv1D(v_batch_size,3, activation='relu', padding='same')(x1)
        flat = Flatten()(x2)
        
        encoded = Dense(100, activation='relu')(flat)
        cls = Dense(3, activation='softmax')(encoded)

        model= Model(input_sig, cls)

        return model    
    
    def test_model (self, v_df):
        scores_c = []
        scores_w1 = []
        scores_w2 = []
        scores_a = []
        
        for sid in self.ids:
            print("============= test subject " +str(sid)+ " ==================")
            x_train, y_train, x_test, y_test, yk, yk_test = self.get_data (test_id = sid,
                                                                           v_batch_size=sf_chest,
                                                                           v_feat_list=feat_sf700, 
                                                                           df=self.df_c)
            x_trainw1, y_trainw1, x_testw1, y_testw1, yk1w1, yk_test1w1 = self.get_data (test_id = sid,
                                                                           v_batch_size=sf_BVP,
                                                                           v_feat_list=feat_sf64, 
                                                                           df=self.df_w1)
            x_trainw2, y_trainw2, x_testw2, y_testw2, yk2w2, yk_test2w2 = self.get_data (test_id = sid,
                                                                           v_batch_size=sf_EDA,
                                                                           v_feat_list=feat_sf4, 
                                                                           df=self.df_w2)

           
            x_train_a, y_train_a, x_test_a, y_test_a, yk_a, yk_test_a = self.get_data (test_id =sid, 
                                                                           v_batch_size=4, 
                                                                           v_feat_list=feat_all, 
                                                                           df=v_df)
            
            modelw1 = self.cnn_model_w1(v_batch_size=sf_BVP, n_feat=len(feat_sf64))
            modelw1.compile(optimizer=RMSprop(lr=0.00025), loss="categorical_crossentropy")
            history = modelw1.fit(x_trainw1, y_trainw1, epochs=4)

            pred_trainw1 = modelw1.predict(x_trainw1)
            pred_testw1 = modelw1.predict(x_testw1)
            
            print("pred_trainw1.shape: ", pred_trainw1.shape)
            print("pred_testw1.shape: ", pred_testw1.shape)
            print("y_testw1.shape: ", y_testw1.shape)
            
            acc_w1 = accuracy_score(np.argmax(y_testw1, axis=1), np.argmax(pred_testw1, axis=1))
            f1_w1 = f1_score(np.argmax(y_testw1, axis=1), np.argmax(pred_testw1, axis=1), average='weighted')
            

            modelw2 = self.cnn_model_w2(v_batch_size=sf_EDA, n_feat=len(feat_sf4))
            modelw2.compile(optimizer=RMSprop(lr=0.00025), loss="categorical_crossentropy")
            history = modelw2.fit(x_trainw2, y_trainw2, epochs=4)

            pred_trainw2 = modelw2.predict(x_trainw2)
            pred_testw2 = modelw2.predict(x_testw2)
            acc_w2 = accuracy_score(np.argmax(y_testw2, axis=1), np.argmax(pred_testw2, axis=1))
            f1_w2 = f1_score(np.argmax(y_testw2, axis=1), np.argmax(pred_testw2, axis=1), average='weighted')

            m_name = "trained_models/c/cnn_loso"+str(sid)+".h5"
            cnn_m = tf.keras.models.load_model(m_name)
            print("loaded: ", m_name)

            pred_train = cnn_m.predict(x_train)
            pred_test = cnn_m.predict(x_test)
            acc_c = accuracy_score(np.argmax(y_test, axis=1), np.argmax(pred_test, axis=1))
            f1_c = f1_score(np.argmax(y_test, axis=1), np.argmax(pred_test, axis=1), average='weighted')


           
            print("pred_trainw2.shape: ", pred_trainw2.shape)
            print("pred_train.shape: ", pred_train.shape)

            print("pred_testw2.shape: ", pred_testw2.shape)
            print("pred_test.shape: ", pred_test.shape)
            
            #cm = confusion_matrix(y_test, pred_test)

            print("subject ", sid)
            print("chest accuracy: ", acc_c)
            print("chest f1 score: ", f1_c)
            print("")
            
            m_name = "trained_models/c/cnn_all_loso"+str(sid)+".h5"
            cnn_m2 = tf.keras.models.load_model(m_name)
            print("loaded: ", m_name)

            pred_train2 = cnn_m2.predict(x_train_a)
            pred_test2 = cnn_m2.predict(x_test_a)
            acc_a = accuracy_score(np.argmax(y_test_a, axis=1), np.argmax(pred_test2, axis=1))
            f1_a = f1_score(np.argmax(y_test_a, axis=1), np.argmax(pred_test2, axis=1), average='weighted')

            scores_c.append([sid, acc_c, f1_c])
            scores_w1.append([sid, acc_w1, f1_w1])
            scores_w2.append([sid, acc_w2, f1_w2])
            scores_a.append([sid, acc_a, f1_a])
        
        return scores_c, scores_w1, scores_w2, scores_a


In [5]:
cnn_m = cnn_model ()
cnn_m.train_model_c ()

merged train: (31029, 5, 700) (31029, 3)
merged test : (2121, 5, 700) (2121, 3)


C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
970/970 [==============================] - 141s 131ms/step - loss: 0.4464
Epoch 2/10
970/970 [==============================] - 127s 131ms/step - loss: 0.3119
Epoch 3/10
970/970 [==============================] - 128s 132ms/step - loss: 0.2930
Epoch 4/10
970/970 [==============================] - 123s 127ms/step - loss: 0.2552
Epoch 5/10
970/970 [==============================] - 122s 126ms/step - loss: 0.2367
Epoch 6/10
970/970 [==============================] - 126s 130ms/step - loss: 0.2016
Epoch 7/10
970/970 [==============================] - 125s 129ms/step - loss: 0.1856
Epoch 8/10
970/970 [==============================] - 122s 126ms/step - loss: 0.1702
Epoch 9/10
970/970 [==============================] - 123s 127ms/step - loss: 0.1572
Epoch 10/10
970/970 [==============================] - 124s 127ms/step - loss: 0.1439
saved  trained_models/c/cnn_loso2.h5
merged train: (30996, 5, 700) (30996, 3)
merged test : (2154, 5, 700) (2154, 3)


C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
969/969 [==============================] - 127s 129ms/step - loss: 0.4626
Epoch 2/10
969/969 [==============================] - 134s 139ms/step - loss: 0.3027
Epoch 3/10
969/969 [==============================] - 141s 145ms/step - loss: 0.2708
Epoch 4/10
969/969 [==============================] - 128s 132ms/step - loss: 0.2332
Epoch 5/10
969/969 [==============================] - 123s 127ms/step - loss: 0.1976
Epoch 6/10
969/969 [==============================] - 122s 126ms/step - loss: 0.1717
Epoch 7/10
969/969 [==============================] - 123s 127ms/step - loss: 0.1546
Epoch 8/10
969/969 [==============================] - 121s 125ms/step - loss: 0.1493
Epoch 9/10
969/969 [==============================] - 123s 127ms/step - loss: 0.1257
Epoch 10/10
969/969 [==============================] - 121s 125ms/step - loss: 0.1231
saved  trained_models/c/cnn_loso3.h5
merged train: (30985, 5, 700) (30985, 3)
merged test : (2165, 5, 700) (2165, 3)


...

Epoch 1/10
967/967 [==============================] - 141s 142ms/step - loss: 0.4838
Epoch 2/10
967/967 [==============================] - 128s 132ms/step - loss: 0.3228
Epoch 3/10
967/967 [==============================] - 130s 135ms/step - loss: 0.2802
Epoch 4/10
967/967 [==============================] - 132s 136ms/step - loss: 0.2527
Epoch 5/10
967/967 [==============================] - 128s 132ms/step - loss: 0.2129
Epoch 6/10
967/967 [==============================] - 129s 133ms/step - loss: 0.1945
Epoch 7/10
967/967 [==============================] - 124s 128ms/step - loss: 0.1748
Epoch 8/10
967/967 [==============================] - 124s 128ms/step - loss: 0.1653
Epoch 9/10
967/967 [==============================] - 124s 129ms/step - loss: 0.1492
Epoch 10/10
967/967 [==============================] - 124s 129ms/step - loss: 0.1396
saved  trained_models/c/cnn_loso15.h5
merged train: (30929, 5, 700) (30929, 3)
merged test : (2221, 5, 700) (2221, 3)


C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
967/967 [==============================] - 130s 132ms/step - loss: 0.5101
Epoch 2/10
967/967 [==============================] - 128s 132ms/step - loss: 0.3403
Epoch 3/10
967/967 [==============================] - 125s 129ms/step - loss: 0.2923
Epoch 4/10
967/967 [==============================] - 126s 131ms/step - loss: 0.2487
Epoch 5/10
967/967 [==============================] - 125s 129ms/step - loss: 0.2118
Epoch 6/10
967/967 [==============================] - 125s 129ms/step - loss: 0.1950
Epoch 7/10
967/967 [==============================] - 131s 136ms/step - loss: 0.1752
Epoch 8/10
967/967 [==============================] - 124s 128ms/step - loss: 0.1568
Epoch 9/10
967/967 [==============================] - 132s 136ms/step - loss: 0.1419
Epoch 10/10
967/967 [==============================] - 127s 131ms/step - loss: 0.1283
saved  trained_models/c/cnn_loso16.h5
merged train: (30874, 5, 700) (30874, 3)
merged test : (2276, 5, 700) (2276, 3)


C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
965/965 [==============================] - 129s 132ms/step - loss: 0.4684
Epoch 2/10
965/965 [==============================] - 123s 128ms/step - loss: 0.3322
Epoch 3/10
965/965 [==============================] - 124s 128ms/step - loss: 0.3000
Epoch 4/10
965/965 [==============================] - 124s 129ms/step - loss: 0.2482
Epoch 5/10
965/965 [==============================] - 124s 129ms/step - loss: 0.2106
Epoch 6/10
965/965 [==============================] - 124s 129ms/step - loss: 0.1997
Epoch 7/10
965/965 [==============================] - 128s 132ms/step - loss: 0.1700
Epoch 8/10
965/965 [==============================] - 128s 133ms/step - loss: 0.1549
Epoch 9/10
965/965 [==============================] - 126s 130ms/step - loss: 0.1460
Epoch 10/10
965/965 [==============================] - 125s 129ms/step - loss: 0.1324
saved  trained_models/c/cnn_loso17.h5


In [24]:
cnn_m = cnn_model ()
cnn_m.train_model_all (df)
s, s1, s2, s3 = cnn_m.test_model(df)

merged train: (31029, 8, 4) (31029, 3)
merged test : (2121, 8, 4) (2121, 3)


C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10
970/970 [==============================] - 5s 2ms/step - loss: 0.6348A: 0
Epoch 2/10
970/970 [==============================] - 1s 1ms/step - loss: 0.3573
Epoch 3/10
970/970 [==============================] - 2s 2ms/step - loss: 0.3111
Epoch 4/10
970/970 [==============================] - 1s 1ms/step - loss: 0.2695
Epoch 5/10
970/970 [==============================] - 1s 2ms/step - loss: 0.2268
Epoch 6/10
970/970 [==============================] - 1s 1ms/step - loss: 0.1882
Epoch 7/10
970/970 [==============================] - 1s 1ms/step - loss: 0.1549
Epoch 8/10
970/970 [==============================] - 1s 1ms/step - loss: 0.1325
Epoch 9/10
970/970 [==============================] - 1s 1ms/step - loss: 0.1127
Epoch 10/10
970/970 [==============================] - 1s 1ms/step - loss: 0.0969
saved  trained_models/c/cnn_all_loso2.h5
merged train: (30996, 8, 4) (30996, 3)
merged test : (2154, 8, 4) (2154, 3)
Epoch 1/10


C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


969/969 [==============================] - 2s 2ms/step - loss: 0.6026
Epoch 2/10
969/969 [==============================] - 1s 1ms/step - loss: 0.3153
Epoch 3/10
969/969 [==============================] - 1s 1ms/step - loss: 0.2647
Epoch 4/10
969/969 [==============================] - 1s 1ms/step - loss: 0.2275
Epoch 5/10
969/969 [==============================] - 1s 1ms/step - loss: 0.1945
Epoch 6/10
969/969 [==============================] - 1s 2ms/step - loss: 0.1729
Epoch 7/10
969/969 [==============================] - 1s 1ms/step - loss: 0.1544
Epoch 8/10
969/969 [==============================] - 1s 1ms/step - loss: 0.1407
Epoch 9/10
969/969 [==============================] - 1s 2ms/step - loss: 0.1292
Epoch 10/10
969/969 [==============================] - 1s 2ms/step - loss: 0.1119
saved  trained_models/c/cnn_all_loso3.h5
merged train: (30985, 8, 4) (30985, 3)
merged test : (2165, 8, 4) (2165, 3)
Epoch 1/10


...

Epoch 1/4
967/967 [==============================] - 5s 2ms/step - loss: 0.9476
Epoch 2/4
967/967 [==============================] - 2s 2ms/step - loss: 0.8722
Epoch 3/4
967/967 [==============================] - 2s 2ms/step - loss: 0.8352
Epoch 4/4
967/967 [==============================] - 2s 2ms/step - loss: 0.8231
pred_trainw1.shape:  (30919, 3)
pred_testw1.shape:  (2233, 3)
y_testw1.shape:  (2233, 3)
Epoch 1/4
967/967 [==============================] - 2s 1ms/step - loss: 1.0290
Epoch 2/4
967/967 [==============================] - 1s 1ms/step - loss: 0.7075
Epoch 3/4
967/967 [==============================] - 1s 1ms/step - loss: 0.6578
Epoch 4/4
967/967 [==============================] - 1s 1ms/step - loss: 0.6141
loaded:  trained_models/c/cnn_loso15.h5
pred_trainw2.shape:  (30918, 3)
pred_train.shape:  (30917, 3)
pred_testw2.shape:  (2233, 3)
pred_test.shape:  (2233, 3)
subject  15
chest accuracy:  0.7617554858934169
chest f1 score:  0.7244930428127334

loaded:  trained_models/c/

C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/4
967/967 [==============================] - 5s 2ms/step - loss: 0.9644
Epoch 2/4
967/967 [==============================] - 2s 2ms/step - loss: 0.8779
Epoch 3/4
967/967 [==============================] - 2s 2ms/step - loss: 0.8620
Epoch 4/4
967/967 [==============================] - 2s 2ms/step - loss: 0.8473
pred_trainw1.shape:  (30931, 3)
pred_testw1.shape:  (2221, 3)
y_testw1.shape:  (2221, 3)
Epoch 1/4
967/967 [==============================] - 2s 1ms/step - loss: 0.8102
Epoch 2/4
967/967 [==============================] - 1s 1ms/step - loss: 0.6118
Epoch 3/4
967/967 [==============================] - 1s 1ms/step - loss: 0.5746
Epoch 4/4
967/967 [==============================] - 1s 1ms/step - loss: 0.5447
loaded:  trained_models/c/cnn_loso16.h5
pred_trainw2.shape:  (30930, 3)
pred_train.shape:  (30929, 3)
pred_testw2.shape:  (2221, 3)
pred_test.shape:  (2221, 3)
subject  16
chest accuracy:  0.9482215218370104
chest f1 score:  0.9454650434193289

loaded:  trained_models/c/

C:\Users\srovi\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/4
965/965 [==============================] - 4s 2ms/step - loss: 0.9522
Epoch 2/4
965/965 [==============================] - 2s 2ms/step - loss: 0.8943
Epoch 3/4
965/965 [==============================] - 2s 2ms/step - loss: 0.8641
Epoch 4/4
965/965 [==============================] - 2s 2ms/step - loss: 0.8524
pred_trainw1.shape:  (30876, 3)
pred_testw1.shape:  (2276, 3)
y_testw1.shape:  (2276, 3)
Epoch 1/4
965/965 [==============================] - 4s 1ms/step - loss: 0.9310
Epoch 2/4
965/965 [==============================] - 1s 1ms/step - loss: 0.5892
Epoch 3/4
965/965 [==============================] - 1s 1ms/step - loss: 0.5406
Epoch 4/4
965/965 [==============================] - 1s 1ms/step - loss: 0.5236
loaded:  trained_models/c/cnn_loso17.h5
pred_trainw2.shape:  (30875, 3)
pred_train.shape:  (30874, 3)
pred_testw2.shape:  (2276, 3)
pred_test.shape:  (2276, 3)
subject  17
chest accuracy:  0.820738137082601
chest f1 score:  0.7792669714604291

loaded:  trained_models/c/c

In [25]:
np.array(s)[:,1].mean() #accuracy chest

0.7946811467383398

In [26]:
np.array(s)[:,1].std()

0.12262566864558332

In [27]:
np.array(s)[:,2].mean() #f1-score chest

0.7839396297390945

In [28]:
np.array(s)[:,2].std()

0.11805498477846282

In [29]:
np.array(s1)[:,1].mean() #accuracy wrist BVP

0.5750948184244862

In [30]:
np.array(s1)[:,1].std()

0.10188267321739736

In [31]:
np.array(s1)[:,2].mean() #f1-score wrist BVP

0.5236772655601138

In [32]:
np.array(s1)[:,2].std()

0.08797760311046976

In [33]:
np.array(s2)[:,1].mean() #accuracy wrist EDA, TEMP

0.698523570072321

In [34]:
np.array(s2)[:,1].std()

0.17514120994796814

In [35]:
np.array(s2)[:,2].mean() #f1-score wrist EDA, TEMP

0.6544770623778784

In [36]:
np.array(s2)[:,2].std()

0.16938917978062748

In [38]:
np.array(s3)[:,1].mean() #accuracy all modalities combined

0.8382879539468207

In [39]:
np.array(s3)[:,1].std()

0.14296138686270235

In [40]:
np.array(s3)[:,2].mean() #f1-score all

0.825644619440786

In [41]:
np.array(s)[:,2].std()

0.11805498477846282